# Base

In [30]:
import ete3
from ete3 import Tree
import os
import os
import glob
import numpy as np
from Bio import AlignIO 
import random
import re
import copy
from sympy.solvers import solve
from sympy import *

In [161]:
t = Tree("(((A:0.687, B:0.601):1.244,(C:0.804,(D:0.015, E:0.017):0.79):0.333):0.555,F:1.001);")
#t = Tree("((A:0.011, B:0.012):0.700,C:0.543);")

In [3]:
os.getcwd()
os.chdir("./Alignments/block_aln/")

# Getting block-alignment

In [ ]:
for file in glob.glob("*.fasta"):
    with open(file, 'r') as fname:
        matrix = np.array([np.array(rec.seq) for rec in AlignIO.read(fname, "fasta")])
        names = [rec.name for rec in AlignIO.read("PF00004_1.fasta", "fasta")]
        indexes = [num for num in range(matrix.shape[1]) if list(matrix[:,num]).count("-") > len(matrix[:,num]) * 0.5]
        matrix = np.delete(matrix, indexes, axis=1)
    with open("./block_aln/"+file.split(".")[0]+"_block.fasta", 'w') as fname:
        for name, seq in zip(names, list(matrix)):
            fname.write(">" + name + '\n' + ''.join(list(seq)) + '\n')

# New_way

In [4]:
class nd:
    def __init__(self, seq):
        self.seq = seq
    def children(self, children):
        self.children = children
    def dist(self, dist):
        self.dist = dist
    def name(self, name):
        self.name = name

In [162]:
class parser(nd):
    def __init__(self, x, tree):
        self.x = x
        self.t = tree
        super().__init__(0)

    #Choose random alignment (with seq_len more than self.x)
    def random_aln(self):
        while True:
            file = random.choice([file for file in glob.glob("*.fasta")])
            aln = AlignIO.read(open(file, 'r'), "fasta")
            if len(str(aln[0].seq)) >= self.x:
                break
        self.file = file
        self.alignment_dict = {record.id:str(record.seq) for record in aln}
        
    #Make root sequence and pin columns   
    def root_seq(self):
        column_id = [random.randint(0, self.x) for _ in range(self.x)]
        a = AlignIO.read(open(self.file, 'r'), "fasta")
        names, seq = [],''
        for num in range(self.x):
            record = random.randint(0, len(a)-1)
            names.append(a[record].id)
            seq += str(a[record].seq)[column_id[num]] 
        self.columns = column_id
        self.seq = seq
        self.names = names
             
    #Make all_dists and all_id lists
    def dists(self):
        all_id, all_dists, values, one_id = [], [], [], ''
        with open(self.file[:-12]+".dist", 'r') as fname:
            for string in fname:
                if re.search("[A-Za-z]+", string) != None:
                    all_id.append(one_id)
                    all_dists.append(values)
                    one_id, values = string.split()[0], string.split()[1:]
                else:
                    values += string.split()
            all_id.append(one_id)
            all_dists.append(values)
        self.all_id = all_id[1:]
        self.all_dists = all_dists[1:]
        
    #NORMALIZATION
    def normal(self):
        container = []
        tree_max = 1.244
        for subset in self.all_dists:
            dist_max = max([float(num) for num in subset])
            dist_min = 10
            for i in subset:
                if float(i) < float(dist_min) and float(i) != float(0):
                    dist_min = i
            dist_diff = float(dist_max) - float(dist_min)
            alpha = dist_diff / (1.244 - 0.015)
            container.append(list(map(lambda x: str(float(x)*alpha), subset)))
        #print(container)
        self.all_dists = container
            
    #Make sequences
    def make_sample(self):
        nodes, graph, lvl = [], [], -1
        
        checker = []
        for node in self.t.traverse("levelorder"):
            for i in list(map(lambda x: x.dist, node.children)):
                graph.append([node.dist, i])        
        for node in self.t.traverse("levelorder"):
            for i in list(map(lambda x: x.dist, node.children)):
                checker.append([node.dist, i])        
        self.graph = graph  
        self.checker = checker
        lvl = 0
        while self.graph[lvl][0] == 0.0:
            self.graph[lvl][0] = self.names 
            #self.checker[lvl][0] = self.seq
            lvl += 1  
        check_point_dists = copy.deepcopy(self.all_dists)
        prev = ''
        
        for item in self.graph:
            if item[0] != prev:
                self.all_dists = copy.deepcopy(check_point_dists)
            else:
                for i in self.all_dists:
                    pass
            self.names = item[0]
            dist = float(item[1])
            seq = ''
            names = []
            less, more, less_index, more_index = 100, 100, [], [] #0, 0      
            
            #################################################################################
            # Generating sequence
            for al_num in range(len(self.names)):
                values = self.all_dists[self.all_id.index(self.names[al_num])]
                less, more = 100, 100
                for i in range(len(values)):
                    if float(values[i]) <= dist and abs(float(values[i])-dist) < abs(float(less)-dist):
                        less = values[i]
                    elif float(values[i]) > dist and abs(float(values[i])-dist) < abs(float(more)-dist):
                        more = values[i]
                
                
                
                
                if less != 100 and more != 100:
                    
                    diff1 = (round(abs(dist-float(less)), 4), less)
                    diff2 = (round(abs(dist-float(more)),4), more)
                    
                    b1,b2 = min(diff1[0], diff2[0]), max(diff1[0],diff2[0])
                    x = Symbol('x')
                    prob = round(solve((b1 + b2)*x-b1,x)[0],4)
                    our_dist = np.random.choice([b1,b2], 1, p = [1-prob, prob])
                    
                    if our_dist == diff1[0]:
                        our_dist = less
                    elif our_dist == diff2[0]:
                        our_dist = more
                    
 
                    
                else:
                    if less != 100: our_dist = less
                    elif more != 100: our_dist = more
                
                print(our_dist, dist)
                #print(values)
                ind = random.choice([i for i, x in enumerate(values) if x == our_dist])
                #self.all_dists[self.all_dists.index(values)][ind] = '1000'
                names.append(self.all_id[ind])
                seq += self.alignment_dict[self.all_id[ind]][self.columns[al_num]]
            i = 0
            while True:
                if i == len(graph):
                    break
                if self.graph[i][0] == dist:
                    break
                i += 1
            while True:
                if i == len(graph):
                    break
                if self.graph[i][0] != dist:
                    break
                if self.graph[i][0] == dist:
                    self.graph[i][0] = names
                    #self.checker[i][0] = seq
                i += 1
            print(seq, item[1])
            #print(self.graph)
            nodes.append(nd(seq))
            prev = item[0]
        self.sample = nodes
        
    # Appending information of dist
    def app_inf(self):
        j = 0
        for node in self.t.traverse("levelorder"):
            for i in list(map(lambda x:x.dist, node.children)):
                self.sample[j].dist = float(i)
                j += 1
        for node in self.sample:
            anchor = node.dist
            node.children = []
            for pair in self.checker:
                if anchor == pair[0]:
                    node.children.append(pair[1])
        for node in self.sample:
            for i in range(len(node.children)):
                for child in self.sample:
                    if node.children[i] == child.dist:
                        node.children[i] = child.seq
        j = 0
        for item in self.sample:
            item.name = ''
        for node in self.t.iter_descendants("levelorder"):
            for item in self.sample:
                if item.dist == node.dist:
                    item.name = node.name
        #print('DONE')

In [163]:
f = parser(100, t) 
f.random_aln()
f.file
f.root_seq()
f.seq
f.dists()
#f.normal()
f.seq
f.make_sample()
f.app_inf()

0.492006 0.555
0.000000 0.555
0.670538 0.555
1.229291 0.555
0.705813 0.555
0.717360 0.555
0.428623 0.555
0.754614 0.555
0.705813 0.555
0.526579 0.555
0.458112 0.555
0.546240 0.555
0.474608 0.555
0.886280 0.555
0.474608 0.555
0.570973 0.555
0.000000 0.555
0.526579 0.555
0.495642 0.555
0.000000 0.555
0.711901 0.555
0.000000 0.555
0.573680 0.555
0.546240 0.555
0.552786 0.555
0.604500 0.555
0.573680 0.555
0.550548 0.555
0.550548 0.555
0.470190 0.555
0.428623 0.555
0.000000 0.555
0.000000 0.555
0.810757 0.555
0.573680 0.555
0.824221 0.555
0.670538 0.555
0.886280 0.555
0.000000 0.555
0.570973 0.555
0.537164 0.555
0.705813 0.555
0.547314 0.555
0.702678 0.555
0.428623 0.555
0.492006 0.555
0.447378 0.555
0.492006 0.555
0.458112 0.555
0.402470 0.555
0.547168 0.555
0.859166 0.555
0.754614 0.555
0.428623 0.555
0.702049 0.555
0.502421 0.555
0.547168 0.555
0.886280 0.555
1.229291 0.555
0.670538 0.555
0.495642 0.555
0.550548 0.555
0.573680 0.555
0.552786 0.555
0.418275 0.555
0.550548 0.555
0.754614 0

0.731239 0.601
0.418275 0.601
0.832419 0.601
0.000000 0.601
0.803100 0.601
0.418275 0.601
0.800203 0.601
0.418275 0.601
1.194094 0.601
0.634162 0.601
0.428623 0.601
0.754614 0.601
0.810757 0.601
0.661706 0.601
0.573680 0.601
1.194094 0.601
0.418275 0.601
0.474608 0.601
0.604546 0.601
0.411531 0.601
1.194094 0.601
0.634162 0.601
0.428623 0.601
0.661706 0.601
0.731239 0.601
0.859166 0.601
0.760821 0.601
0.754614 0.601
0.547168 0.601
0.754614 0.601
0.705813 0.601
0.766423 0.601
0.573680 0.601
0.418275 0.601
0.573680 0.601
0.705813 0.601
0.492006 0.601
0.763469 0.601
0.634162 0.601
0.604763 0.601
0.805964 0.601
0.705813 0.601
0.670538 0.601
0.754614 0.601
0.604763 0.601
0.886280 0.601
0.711901 0.601
0.670538 0.601
0.573680 0.601
0.661706 0.601
1.065079 0.601
0.805964 0.601
0.000000 0.601
0.547168 0.601
0.418275 0.601
0.547168 0.601
1.065079 0.601
0.670538 0.601
0.550548 0.601
0.763469 0.601
0.573680 0.601
1.194094 0.601
0.492006 0.601
0.763469 0.601
0.634162 0.601
0.754614 0.601
0.000000 0

In [164]:
with open('../../out'+str(1)+'.fasta', 'w') as out:
    for item in f.sample:
        if item.name != '':
            out.write('>'+str(item.name)+'\n'+str(item.seq)+'\n')

In [38]:
[('0.671414', 0.0286), ('0.827183', 0.1272), 0.7] ['0.827183'] [0.8164, 0.1836]
np.random.choice([a1, a2], 1, p=[1 - prob, prob])

0.689349 0.7
0.700488 0.7
0.718968 0.7
0.723890 0.7
0.679016 0.7
0.706491 0.7
0.703000 0.7
0.710591 0.7
0.702959 0.7
0.684606 0.7
0.694670 0.7
0.708258 0.7
0.687984 0.7
0.708258 0.7
0.702089 0.7
0.598309 0.7
0.676730 0.7
0.706840 0.7
0.705832 0.7
0.698454 0.7
0.702513 0.7
0.733972 0.7
0.681155 0.7
0.695138 0.7
0.700406 0.7
0.581747 0.7
0.626611 0.7
0.702050 0.7
0.699308 0.7
0.705358 0.7
0.676765 0.7
0.671697 0.7
0.704101 0.7
0.697331 0.7
0.697949 0.7
1.345598 0.7
0.740500 0.7
0.693883 0.7
0.688004 0.7
0.712413 0.7
0.691099 0.7
0.740000 0.7
0.685593 0.7
0.772815 0.7
0.685373 0.7
0.717108 0.7
0.687259 0.7
0.684606 0.7
0.711909 0.7
0.674536 0.7
0.665595 0.7
1.445652 0.7
0.673176 0.7
0.710430 0.7
0.576017 0.7
0.715570 0.7
0.688344 0.7
0.662935 0.7
0.666374 0.7
0.698825 0.7
0.706840 0.7
0.700406 0.7
0.702612 0.7
0.806286 0.7
0.704544 0.7
1.158150 0.7
0.692011 0.7
0.699701 0.7
0.000000 0.7
0.672500 0.7
0.689346 0.7
0.659034 0.7
0.700488 0.7
0.687847 0.7
0.676797 0.7
0.687847 0.7
0.697331 0.7

In [35]:
re.search('.*', 'agdfs')

<_sre.SRE_Match object; span=(0, 5), match='agdfs'>

In [34]:
import re

In [117]:
for _ in range(10):
    print(np.random.choice([0,1], 1, p=[0.8,0.2]))

[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]


In [165]:
a = [-1,4,5]

TypeError: '<' not supported between instances of 'list' and 'int'